# TAREA 3 - ETL DATASET BI

## INTEGRANTES:
- Apolonio Cuevas Manuel Alberto
- Sansores Arjona Alejandro Jesus
- Israel Alejandro Cel Alcocer
- Salvador Iram Salas Baez
- Leo Sebastian Contreras Raymund

 ## Configuracion del Ambiente


In [ ]:
---> set the Role
USE ROLE SNOWFLAKE_LEARNING_ROLE;
---> set the Warehouse
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;

USE DATABASE INFO_EMPLEADOS;

In [ ]:
import toml
from snowflake.snowpark.session import Session

# Ruta al archivo connections.toml dentro del stage del notebook
# Nota: La ruta es relativa al stage, por lo general es el nombre del archivo.
connections_file_path = "connections.toml"

# Leer el archivo TOML y cargar las configuraciones
try:
    with open(connections_file_path, "r") as f:
        config = toml.load(f)
    print("Archivo connections.toml cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo {connections_file_path} no fue encontrado.")
    config = None

# Crear la sesión si el archivo se cargó correctamente
if config:
    try:
        session = Session.builder.configs(config).create()
        print("Sesión de Snowflake creada exitosamente.")
    except Exception as e:
        print(f"Error al crear la sesión: {e}")
else:
    print("No se puede crear la sesión debido a un error en la carga de la configuración.")

# Opcional: Mostrar detalles de la sesión
if 'session' in locals():
    print("Database:", session.get_current_database())
    print("Schema:", session.get_current_schema())
    print("Warehouse:", session.get_current_warehouse())
    print("Role",session.get_current_role())

In [ ]:
# Import python packages
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
from snowflake.snowpark import Session

session = Session.builder.config("connection_name", "myconnection").create()

print(session)

### Cargo de datos en la tabla

In [ ]:
# Importar la librería de Snowpark
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
from snowflake.snowpark.context import get_active_session

# 1. Obtener la sesión activa de Snowpark (¡esto ya lo tienes y es perfecto!)
session = get_active_session()

# 1. Define la tabla de destino
nombre_tabla = "TAREA3_RAW"

# 2. Define la RUTA CORRECTA al archivo en el stage del notebook
# La ruta debe empezar con @ y los nombres de la base de datos, esquema y notebook van entre comillas.
stage_path = '@"INFO_EMPLEADOS"."PUBLIC"."TAREA3"/versions/live/bi.csv'

# 3. Lee el archivo CSV desde el stage
df = session.read.option("encoding", "ISO-8859-1") \
                 .option("header", "true") \
                 .option("infer_schema", "true") \
                 .csv(stage_path)

# 4. Trunca la tabla y carga los nuevos datos
df.write.mode("overwrite").save_as_table(nombre_tabla)

# 5. Muestra una confirmación y una vista previa
print(f"La tabla '{nombre_tabla}' ha sido truncada y cargada con éxito.")
df.show()

In [ ]:
TRUNCATE TABLE TAREA3_RAW;

COPY INTO TAREA3_RAW
FROM snow://notebook/INFO_EMPLEADOS.PUBLIC.TAREA3/versions/live/bi.csv
FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = ',' SKIP_HEADER = 1 ENCODING = 'ISO-8859-1')
ON_ERROR = 'CONTINUE';

SELECT *
FROM TAREA3_RAW;

In [ ]:
import pandas as pd
import os

data = pd.read_csv("bi.csv",encoding="latin-1")
data.head()

In [ ]:
import os
import json
from snowflake.snowpark.context import get_active_session

# 1. Obtener la sesión activa de Snowpark (¡esto ya lo tienes y es perfecto!)
session = get_active_session()

# 2. Cargar las credenciales desde el Snowflake Secret
kaggle_credentials = session.get_secret_value("mi_secreto_kaggle")

# El secreto es un string JSON, así que lo convertimos a un diccionario de Python
kaggle_secrets = json.loads(kaggle_credentials)

# 3. Configurar las variables de entorno para la API de Kaggle
os.environ['KAGGLE_USERNAME'] = kaggle_secrets['username']
os.environ['KAGGLE_KEY'] = kaggle_secrets['key']

# 4. Conectar a Kaggle (esto no cambia)
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

print("¡Conexión exitosa a Kaggle usando Snowflake Secrets!")

In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")